<font size=5>Simple

In [2]:
from helper_dataprocessing import *
import os
import glob
# import warnings
# warnings.filterwarnings('ignore') 
# https://us02web.zoom.us/j/7410331468?pwd=b21CSWloRURiTWVpQjVGd0V3ZXhGUT09

# Step 1: Load Data

The version of Ecotox dates back to 06/11/2020. We used three files:

[i] species.txt

[ii] tests.txt

[iii] results.txt

In the first file there are information regarding the species of animals: all the taxonomic denomination (from domain to variety), other typical denominations (Latin name or common name), and also the ecotox group (fish, mammalians, birds, algae , ...).

In the second file, tests.txt, there are information regarding the tests: laboratory conditions (exposure, control, application frequency, ...), CASRN of the chemical compound and the tested animal. Each cross-table reference is based on internal values that are unnecessary for the models.

The last file contains experiment records: LC50, ACC, EC50. Aggregation of information tables on chemicals, species and tests is based on internal keys.

Similarly, the test results have been merged with the previous ones since the test reference is present in this table.


For information on Melting Point and Water Solubility we used other datasets from CompTox.

In [18]:
DATA_RESULTS_PATH = r"data/raw/results.txt"
DATA_TEST_PATH = r"data/raw/tests.txt"
DATA_SPECIES_PATH = "data/raw/species.txt"
DATA_PROPERTY_PATH = ['data/DSSToxQueryWPred4.xlsx',
 'data/DSSToxQueryWPred3.xlsx',
 'data/DSSToxQueryWPred2.xlsx',
 'data/DSSToxQueryWPred1.xlsx']


keep_columns = ['obs_duration_mean', 'obs_duration_unit',
       'endpoint', 'effect', 'measurement', 'conc1_type', 'conc1_mean',
       'conc1_unit', 'test_cas', 'test_location', 'exposure_type',
       'control_type', 'media_type', 'application_freq_unit', 'class',
       'tax_order', 'family', 'genus', 'species','smiles','mol_weight',
        'melting_point', 'water_solubility']

In [8]:
tests, species, results = load_raw_data(DATA_TEST_PATH, DATA_RESULTS_PATH, DATA_SPECIES_PATH)

tests loaded
Tests table dimensions:  (684920, 122)
species loaded
Species table dimensions:  (27408, 15)
results loaded
Results table dimensions:  (1018565, 137)


In [ ]:
def load_raw_data(DATA_PATH_TESTS, DATA_PATH_RESULTS, DATA_PATH_SPECIES, DATA_PROPERTY_PATH):
    tests = pd.read_csv(DATA_PATH_TESTS, sep = '\|', engine = 'python')
    print('tests loaded')
    print('Tests table dimensions: ', tests.shape)
    species = pd.read_csv(DATA_PATH_SPECIES, sep = '\|', engine = 'python')
    print('species loaded')
    print('Species table dimensions: ', species.shape)
    results = pd.read_csv(DATA_PATH_RESULTS, sep = '\|', engine = 'python')
    print('results loaded')
    print('Results table dimensions: ', results.shape)
    print("start loading the properties data...")
    df_property = pd.DataFrame()
    for i in DATA_PROPERTY_PATH:
        df_property = pd.concat([df_property, pd.read_excel(open(i, 'rb'), # here
                 usecols =  ['Substance_CASRN', 'Structure_SMILES','Structure_MolWt','NCCT_MP', 'NCCT_WS'], engine = 'openpyxl')], axis=0)
        print(str(i)+ "loaded")
    return tests, species, results, df_property

In [ ]:
tests, species, results, properties = load_raw_data(DATA_TEST_PATH, DATA_RESULTS_PATH, DATA_SPECIES_PATH,DATA_PROPERTY_PATH)

In [10]:
df_property = pd.DataFrame()
for i in tqdm(csv_files):
    df_property = pd.concat([df_property, pd.read_excel(open(i, 'rb'), # here
             usecols =  ['Substance_CASRN', 'Structure_SMILES','Structure_MolWt','NCCT_MP', 'NCCT_WS'], engine = 'openpyxl')], axis=0)

In [ ]:
properties = pd.concat([pd.read_excel(open("data/DSSToxQueryWPred1.xlsx", 'rb'), # here
             usecols =  ['Substance_CASRN', 'Structure_SMILES','Structure_MolWt','NCCT_MP', 'NCCT_WS'], engine = 'openpyxl'),
             pd.read_excel(open("data/DSSToxQueryWPred2.xlsx", 'rb'), # here
             usecols =  ['Substance_CASRN', 'Structure_SMILES','Structure_MolWt','NCCT_MP', 'NCCT_WS'], engine = 'openpyxl'),
             pd.read_excel(open("data/DSSToxQueryWPred3.xlsx", 'rb'), # here
             usecols =  ['Substance_CASRN', 'Structure_SMILES','Structure_MolWt','NCCT_MP', 'NCCT_WS'], engine = 'openpyxl'),
             pd.read_excel(open("data/DSSToxQueryWPred4.xlsx", 'rb'), # here
             usecols =  ['Substance_CASRN', 'Structure_SMILES','Structure_MolWt','NCCT_MP', 'NCCT_WS'], engine = 'openpyxl')], axis = 0)
properties = properties.rename({'Substance_CASRN': 'test_cas',  'Structure_SMILES':'smiles','Structure_MolWt':'mol_weight',
                                'NCCT_MP':'melting_point', 'NCCT_WS':'water_solubility'}, axis=1)



# Step 2: Prefiltering and Pre-Aggregation

Once loaded the data, we filter the results on endpoint (we take only EC50 and LC50) and effects (we take only mortality (MOR)). We restrict the animal kingdom to fish only.

Also, we removed embryos tests. whose information is contained in the test table, variable "organism_lifestage".


In [ ]:
def prefilter(species, tests, results,endpoint=None,label="simple",fish_species=None,all_property = 'itself',effect='MOR'):
    results.loc[:, 'effect'] = results.effect.apply(lambda x: x.replace('/','') if '/' in x else x)
    results.loc[:, 'effect'] = results.effect.apply(lambda x: x.replace('~','') if '~' in x else x)
    results.loc[:, 'endpoint'] = results.endpoint.apply(lambda x: x.replace('/','') if '/' in x else x)
    results.loc[:, 'endpoint'] = results.endpoint.apply(lambda x: x.replace('*','') if '*' in x else x)
    if label =='datafusion':
        resc = results[results.endpoint.str.contains('C')].copy()
        if all_property == 'no':
        	rconc = resc.loc[~(results.effect.str.contains(effect)),: ]
        elif all_property =='itself':
            rconc = resc.loc[~((~results.endpoint.str.contains(endpoint)) & (results.effect.str.contains(effect))),:]
        elif all_property == 'all':
        	rconc = resc
        elif all_property == 'other':
            rconc = resc.loc[~(results.endpoint.str.contains(endpoint) & results.effect.str.contains(effect)),: ]
        print('There are', rconc.shape[0], 'tests.(except '+ endpoint +')')
        test = tests.copy()
    elif label == 'simple':
        resc = results[(results.endpoint.str.contains(endpoint))]
        print('There are', resc.shape[0], 'tests.(only '+ endpoint +')')
        rconc = resc[resc.effect.str.contains(effect)]
        print('There are', rconc.shape[0], 'tests consideing about '+ effect +'.')
        test = tests[tests.organism_lifestage != "EM"]
    else:
        rconc = results[results.endpoint.str.contains('C')].copy()
        test = tests[tests.organism_lifestage != "EM"]

    # focus on fishes
    species = species[~species.ecotox_group.isnull()]
    sp_fish = species[species.ecotox_group.str.contains('Fish')]

    # merging tests information and taxanomy information
    test_fish = test.merge(sp_fish, on="species_number")
    print('There are', test_fish.shape[0], 'tests on these fishes.')
    
    # merging with results
    results_prefilter = rconc.merge(test_fish, on = 'test_id')
    print('All merged into one dataframe. Size was', results_prefilter.shape,'.')
    print('The unique chemical number was',len(results_prefilter.test_cas.unique()),'.')
    return results_prefilter

In [16]:
results_prefilter = prefilter(species, tests, results, endpoint="LC50|EC50", effect='MOR')

results_prefilter.loc[:, 'test_cas'] = results_prefilter.test_cas.apply(to_cas)
filtered_results = results_prefilter.merge(properties,on="test_cas")

There are 185540 tests.(only LC50|EC50)
There are 136569 tests consideing about MOR.
There are 155208 tests on these fishes.
All merged into one dataframe. Size was (64460, 272) .
The unique chemical number was 3481 .



# Step 3: Feature Selection and Imputation

The most interesting variables are in the "keep_columns" list (19 variables).

After the selection, I proceeded to impute or remove the missing values.


CONCENTRATION FEAT.
I started this phase with the variables concerning the concentration (conc1_mean is the target variable, of type string).

- If the target variable has a null value, the record is deleted.
- If the target variable has an asterisk "*" within the value, the asterisk is stripped and the value restored.
- If the target variable takes the value "> 100000", this is removed (1 case only).
- The target variable is transformed to a float type.
- If the variable 'conc1_unit', that is the unit of measurement of the concentration, has the letters "AI" (Active Ingredient) within the measure, these are removed, and the unit of measure is restored.
- The values of the target variable are all transformed into a single unit of measure, that is mg / L, applying the necessary transformations.
- The variable 'conc1_unit' (all mg / L) loses meaning and is removed.
- If the type of concentration (Active ingredient, dissolved, formulation, ...) has a Not Coded (NC) or Not Reported value, the entire record is removed.


TEST FEAT.
The variables to impute are: "exposure_type", "test_location", "control_type", "media_type" and "application_freq_unit".

In order, by exposure_type:
- Values with the "/" symbol are restored without this symbol.
- The values assuming the "AQUA - NR" mode, ie exposure in water not reported, are attributed with the generic value "AQUA".
- The Not Reported values (about 5000, ie 0.089% of the total) are attributed with the generic value "AQUA" ie exposure in water.

For test_location:
- This variable is removed because it is unbalanced: more than 98% of the total experiments were performed in the laboratory, while the remaining 2% were performed on "artificial" or "natural" soil.

For control_type:
- Values with the "/" symbol are restored without this symbol.
- Created "Unknown" generic class to not discard this variable.

For media_type:
- Values with the "/" symbol are restored without this symbol.
- If this variable takes on a mode between ['NR', 'CUL', 'NONE', 'NC'], where CUL stands for Culture, the entire record is removed.

For application_freq_unit:
- The missing values, Not Reported and Not Coded, are imputed with the most frequent class: "X", ie the concentration of the compound is unitary (the compound is given all at once to the fish).


DURATION FEAT.
For obs_duration_mean, only feature on experiment duration:
- With the help of obs_duration_unit, the experiments all last between [24, 48, 72, 96] hours.


SPECIES FEAT.
- The "species" and "genus" features have null values in the same positions. By deleting the records you get these two neat characteristics.
- Same goes for "family", "tax_order" and "class".


In [19]:
best_db = select_impute_features(filtered_results,keep_columns)

# Step 4: Repeated Experiment Aggregation
Experiments are repeated, but how to consider an experiment as repeated?
We have decided that two experiments are repeated if they share the following characteristics:
- Same fish: I merged all the taxonomic name of the fish into one feature, and created a separate dataset for later aggregation.
- Same compound: ie the CASRN
- Same duration, type of exposure, final check, water in which the fish was tested, frequency of application.

Grouping based on these characteristics, we took the median value and re-aggregated the information regarding the fish.

Endpoint, effect and measurement information is lost. The first two is obvious, all experiments refer to LC50 or EC50, with Mortality effect. "measurement" has no particular relevance given that 99% of the values assume the "MORT" mode, ie mortality. About a hundred experiments were "measured" with survival, "SURV".

At the end of this aggregation there are 28815 experiments (aggregates), with 14 characteristics detected (including the taxonomy aggregation -> "fish").

In [20]:


final_db = repeated_experiments(best_db)

# Step 5: Extraction of SMILES, PubChem2D and molecular descriptors from CASRN
We use the file to extract all information from the CASRN.

If you want to avoid calculating all smiles, pubchems and molecular descriptors from scratch (about 2 hours) you can use the alternative function "process_chemicals" which takes as input a dataset with these info already extracted.


In [39]:
def smiles_to_pubchem(df):

    pubchem = pd.DataFrame()
    pubchem["smiles"] = df["smiles"].unique()
    for i in tqdm(pubchem["smiles"].values):
        try:
            pubchem[pubchem.smiles == i]["pubchem"] = pcp.get_compounds(i, 'smiles')[0].cactvs_fingerprint
        except:
            pubchem[pubchem.smiles == i]["pubchem"] = np.NaN
    df_pubchem = df.merge(pubchem, on="smiles")

    return df_pubchem



In [ ]:
# get the pubchem2d fingerprint
final_pubchem = smiles_to_pubchem(final_db)

  0%|          | 0/2597 [00:00<?, ?it/s]/local/wujimeng/anaconda/envs/rdkit-test/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  0%|          | 7/2597 [00:05<33:55,  1.27it/s]/local/wujimeng/anaconda/envs/rdkit-test/lib/python3.6/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 27%|██▋       | 705/2597 [09:54<25:12,  1.25it/s]

In [46]:
#  use the saved file
pubchem = pd.read_csv("data/cas_property_tot_new.csv")
final_pubchem2 = final_db.merge(pubchem[["smiles","pubchem2d"]],on="smiles")

In [47]:
#  get the properties
final_pubchem = final_pubchem[~final_pubchem.smiles.isnull()]
final_pubchem = final_pubchem[~final_pubchem.pubchem2d.isnull()]
chem_feat = process_chemicals(final_pubchem)

,test_cas,smiles,obs_duration_mean,conc1_type,fish,exposure_type,control_type,media_type,application_freq_unit,conc1_mean,mol_weight,melting_point,water_solubility,class,tax_order,family,genus,species,pubchem2d
0,10102-18-8,[Na+].[Na+].[O-][Se]([O-])=O,96.0,T,Actinopterygii Cypriniformes Cyprinidae Danio ...,R,S,FW,X,23.00,172.947998,NaN,NaN,Actinopterygii,Cypriniformes,Cyprinidae,Danio,rerio,0000000000000000001100000011000000000000000000...
1,10102-18-8,[Na+].[Na+].[O-][Se]([O-])=O,96.0,T,Actinopterygii Cypriniformes Cyprinidae Danio ...,R,S,FW,X,23.00,172.947998,NaN,NaN,Actinopterygii,Cypriniformes,Cyprinidae,Danio,rerio,0000000000000000001100000011000000000000000000...
2,10102-18-8,[Na+].[Na+].[O-][Se]([O-])=O,96.0,T,Actinopterygii Cypriniformes Cyprinidae Danio ...,R,S,FW,X,23.00,172.947998,NaN,NaN,Actinopterygii,Cypriniformes,Cyprinidae,Danio,rerio,0000000000000000001100000011000000000000000000...
3,10102-18-8,[Na+].[Na+].[O-][Se]([O-])=O,48.0,T,Actinopterygii Beloniformes Adrianichthyidae O...,R,Unknown,FW,DLY,4.70,172.947998,NaN,NaN,Actinopterygii,Beloniformes,Adrianichthyidae,Oryzias,latipes,0000000000000000001100000011000000000000000000...
4,10102-18-8,[Na+].[Na+].[O-][Se]([O-])=O,48.0,T,Actinopterygii Beloniformes Adrianichthyidae O...,R,Unknown,FW,DLY,4.70,172.947998,NaN,NaN,Actinopterygii,Beloniformes,Adrianichthyidae,Oryzias,latipes,0000000000000000001100000011000000000000000000...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6932,112-56-1,CCCCOCCOCCSC#N,96.0,A,Actinopterygii Salmoniformes Salmonidae Oncorh...,S,C,FW,X,4.85,203.300003,-9.70489,0.002368,Actinopterygii,Salmoniformes,Salmonidae,Oncorhynchus,clarkii,1110000001110010001100000000000001000000000000...
6933,112-56-1,CCCCOCCOCCSC#N,24.0,A,Actinopterygii Salmoniformes Salmonidae Salvel...,S,C,FW,X,7.05,203.300003,-9.70489,0.002368,Actinopterygii,Salmoniformes,Salmonidae,Salvelinus,namaycush,1110000001110010001100000000000001000000000000...
6934,112-56-1,CCCCOCCOCCSC#N,96.0,A,Actinopterygii Salmoniformes Salmonidae Salvel...,S,C,FW,X,4.00,203.300003,-9.70489,0.002368,Actinopterygii,Salmoniformes,Salmonidae,Salvelinus,namaycush,1110000001110010001100000000000001000000000000...
6935,15541-45-4,[O-][Br](=O)=O,96.0,T,Actinopterygii Salmoniformes Salmonidae Oncorh...,S,I,SW,X,512.00,127.902000,NaN,NaN,Actinopterygii,Salmoniformes,Salmonidae,Oncorhynchus,keta,0000000000000000001100000000000000000000000100...


<font size=5>Datafusion

In [7]:
results_prefilter = prefilter(species, tests, results,'LC50|EC50',label= 'datafusion')
results_prefilter.loc[:, 'test_cas'] = results_prefilter.test_cas.apply(to_cas)
filtered_results = results_prefilter.merge(cas_property,on='test_cas')

There are 426070 tests.(except LC50|EC50)
There are 2079 fish species in total.
There are 163402 tests on these fishes.
All merged into one dataframe. Size was (175414, 272) .
The unique chemical number was 4331 .


In [8]:
ct = pd.crosstab(filtered_results.effect, filtered_results.endpoint)
ct

endpoint,ATCN,BCF,BCFD,BMC02.5,BMC04,BMC05,BMC06,BMC08,BMC10,BMC11,...,IC01,IC10,IC20,IC25,IC50,IC80,LC50,LOEC,MATC,NOEC
effect,,,,,,,,,,,,,,,,,,,,,
ACC,0,9144,256,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2169,5,1562
AVO,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,10,0,0,75,3,104
BCM,0,0,0,0,0,0,0,0,0,0,...,0,4,39,0,27,0,1,8816,6,10427
BEH,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,1,1353,11,2581
CEL,0,0,0,0,0,0,0,0,0,0,...,0,40,45,0,106,2,10,1466,0,1935
DVP,0,0,0,0,0,8,0,0,0,0,...,0,0,0,1,1,0,0,785,77,1193
ENZ,0,0,0,1,1,1,0,1,3,0,...,0,0,51,0,347,0,0,6631,6,7025
FDB,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,610,1,521
GEN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,10,0,0,8983,10,11337


In [14]:
ct = pd.crosstab(filtered_results.effect, filtered_results.endpoint)
dfr = pd.DataFrame()
for j in tqdm(ct.columns):
    for i in ct.index:
        if (ct.loc[i,j] >200):
            pp = filtered_results[filtered_results.endpoint == j]
            pp = pp[pp.effect == i]
            best_db = select_impute_features(pp,keep_columns)
            x = np.array(repeated_experiments(best_db).conc1_mean.unique())
            try:
                final_db = repeated_experiments(best_db)
                final_db.loc[:,'endpoint'] = j
                final_db.loc[:,'effect'] =i
                dfr = pd.concat([dfr, final_db])
            except:
                continue

100%|██████████| 38/38 [00:10<00:00,  3.55it/s]


In [16]:
mol=False
final = dfr.merge(cas_property,on='test_cas',how='left')
final = final[~final.smiles.isnull()]
final = final[~final.pubchem2d.isnull()]
chem_feat = process_chemicals(final)
if mol == True:
    chem_feat['conc1_mean'] = chem_feat['conc1_mean'] *chem_feat['mol_weight']
chem_feat = chem_feat.drop(['melting_unit','mol_weight'],axis=1)

Finding atom number...
Finding number of alone atoms...
Finding single bounds number...
Finding double bounds number...
Finding triple bounds number...
Finding ring number...
Finding mol number...
Finding morgan density...
Finding partition number (LogP)...
Finding number of OH group...
Finding melting point...
Finding water solubility...


In [ ]:
ct = pd.crosstab(chem_feat.effect, chem_feat.endpoint)
df = pd.DataFrame()
df_chem = chem_feat.copy()
old_df = 1
while not (df.equals(old_df)):
    old_df = df
    df = pd.DataFrame()
    for i in tqdm(ct.index):
        for j in ct.columns:
            if (ct.loc[i,j] >200):
                [ths_1,ths_2,ths_3,ths_4] = np.quantile(df_chem[df_chem.effect == i].conc1_mean, [.2,.4,.6,.8])
                pp = df_chem[df_chem.endpoint == j]
                pp = pp[pp.effect == i]
                x = np.array(pp.conc1_mean.unique())
                try:
                    if  np.any(x <= ths_1) & np.any((ths_1 <x) & (x<= ths_2)) & np.any((ths_2 < x) & (x<=ths_3))& np.any((ths_3 < x)&(x <=ths_4))&np.any(x > ths_4):
                            pp.loc[:,'endpoint'] = j
                            pp.loc[:,'effect'] = i
                            df = pd.concat([df, pp])
                except:
                    continue
            else:
                continue
    df_chem = df
    print(df.equals(old_df))

# save the cas property file

In [5]:
pubchem2d = smiles_to_pubchem(cas_property["smiles_cir"].unique())
df = pd.DataFrame()
df["smiles_cir"]=cas_property["smiles_cir"].unique()
df["pub_sim"]=pubchem2d

100%|██████████| 9514/9514 [2:22:28<00:00,  1.11it/s]


In [15]:
df = pd.DataFrame()
df["smiles_cir"]=cas_property["smiles_cir"].unique()
df["pub_sim"]=pubchem2d
df

,smiles_cir,pub_sim
0,NaN,NaN
1,Clc1cc(cc(Cl)c1Cl)c2cc(Cl)c(Cl)c(Cl)c2,1000000001110000000000000000000000000111000000...
2,ClC1(Cl)C2(Cl)C3(Cl)C4(Cl)C(Cl)(Cl)C5(Cl)C3(Cl...,0000000001110000000000000000000000000111100000...
3,[Na+].[O-][N+]([O-])=O,0000000000000010001100000010000000000000000000...
4,CCO[P](=S)(OCC)Oc1ccc(cc1)[N+]([O-])=O,1100000001110010001110000000001001000000000000...
...,...,...
9509,CCCCC/C=C/CC(O)/C=C/C=C/C=C/C(O)CCCC(O)=O,1111000001111000001110000000000000000000000000...
9510,CC[S](=O)(=O)CCS[P](=O)(OC)OC,1100000001100000001110000000001001100000000000...
9511,Oc1cc2Oc3cc(O)c(Cl)cc3[C@]4(OC(=O)c5ccccc45)c2...,1100000001111000001110000000000000000110000000...
9512,C[C@@H]1CN[C@H]2[C@@H](C)[C@@]3(CC[C@H]4[C@@H]...,1111000001111010001100000000000000000000000000...


In [18]:
cas_property=cas_property.merge(df, left_on='smiles_cir', right_on='smiles_cir')

In [24]:
tqdm.pandas()
cas_property["smiles_cir"] = cas_property["test_cas"].progress_apply(find_smiles)

100%|██████████| 12223/12223 [2:46:14<00:00,  1.23it/s]


In [20]:
cas_property.to_csv("data/cas_property_tot_new.csv")

In [37]:
cas_property=cas_property.drop(columns=['smiles', 'pubchem2d', 'melting_point', 'melting_unit','water_solubility'])
cas_property.rename(columns={"smiles_cir":"smiles","pub_sim":"pubchem2d"},inplace=True)